In [1]:
import pandas as pd
import os
from osgeo import ogr
import shapely
import geopandas as gpd
from natcap.invest.recreation.recmodel_client import _polygon_area

In [2]:
def shapely_intersect(aoi_path, intersector_path):

    response_vector_path = (aoi_path)
    response_vector = ogr.Open(response_vector_path)
    response_layer = response_vector.GetLayer()
    response_polygons_lookup = {}  # maps FID to prepared geometry
    for response_feature in response_layer:
        feature_geometry = response_feature.GetGeometryRef()
        feature_polygon = shapely.wkt.loads(feature_geometry.ExportToWkt())
        feature_geometry = None
        response_polygons_lookup[response_feature.GetFID()] = feature_polygon
    response_layer = None

    aoi_intersecting_areas = _polygon_area('area', response_polygons_lookup, intersector_path)
    return(aoi_intersecting_areas)

In [4]:
result = shapely_intersect('../twitter/shp/Priority_Conservation_Areas_current_dissolve_pid.shp', '../slr/CA_MTR23_slr_6ft.shp')

10/03/2017 16:47:49  shapely.geos         INFO     Ring Self-intersection at or near point 545524.50016629195 4221934.62397137
10/03/2017 16:47:49  shapely.geos         INFO     Ring Self-intersection at or near point 579333.70931911503 4210838.9724654602
10/03/2017 16:47:49  shapely.geos         INFO     Ring Self-intersection at or near point 561130.50891981402 4185182.2434437098
10/03/2017 16:47:49  shapely.geos         INFO     Ring Self-intersection at or near point 570783.64324405196 4178793.5145994201
10/03/2017 16:47:49  shapely.geos         INFO     Ring Self-intersection at or near point 575931.55455909495 4166491.2620361499
10/03/2017 16:47:49  shapely.geos         INFO     Ring Self-intersection at or near point 576569.63612843503 4165758.1605388899
10/03/2017 16:47:49  shapely.geos         INFO     Ring Self-intersection at or near point 582170.27333962405 4163180.4517393801
10/03/2017 16:47:49  shapely.geos         INFO     Ring Self-intersection at or near point 577564.3

In [5]:
df = pd.DataFrame(result.items(), columns=['fid', 'area_m2'])

In [6]:
df.to_csv('../pca/pca_area_inundated_slr6ft.csv', index=False)